In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [3]:
data = pd.read_csv('../input/final-sarcasm-dataset/Dataset.csv')
print(data.columns)
data = data[['Emojis','Sarcasm']]
data.head()

Index(['Comments', 'Emojis', 'Text', 'Sarcasm'], dtype='object')


,Emojis,Sarcasm
0,😎,1
1,😂 😂 😂,1
2,😂 😂 😂 😂 😂 😂 😂,1
3,😄,1
4,😂,1


In [4]:
data['Emojis'] = data['Emojis'].astype(str)

print(data[ data['Sarcasm'] == 1].size)
print(data[ data['Sarcasm'] == 0].size)

data.head()

18520
26140


,Emojis,Sarcasm
0,😎,1
1,😂 😂 😂,1
2,😂 😂 😂 😂 😂 😂 😂,1
3,😄,1
4,😂,1


In [5]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features, split=' .,?/-!#&')
tokenizer.fit_on_texts(data['Emojis'].values)
X = tokenizer.texts_to_sequences(data['Emojis'].values)
X = pad_sequences(X)

In [18]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2, name='lstm'), name="Bi-LSTM")),
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.build((None, X.shape[1]))
print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 1, 128)         │       256,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_2             │ (None, 1, 128)         │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Bi-LSTM (Bidirectional)         │ (None, 392)            │       509,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           786 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 766,386 (2.92 MB)

 Trainable params: 766,386 (2.92 MB)

 Non-trainable params: 0 (0.00 B)

None


In [19]:
Y = pd.get_dummies(data['Sarcasm']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(14961, 1) (14961, 2)
(7369, 1) (7369, 2)


In [20]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)

Epoch 1/20
468/468 - 7s - 15ms/step - accuracy: 0.8250 - loss: 0.3758
Epoch 2/20
468/468 - 3s - 7ms/step - accuracy: 0.8578 - loss: 0.3221
Epoch 3/20
468/468 - 3s - 7ms/step - accuracy: 0.8612 - loss: 0.3060
Epoch 4/20
468/468 - 3s - 7ms/step - accuracy: 0.8632 - loss: 0.3001
Epoch 5/20
468/468 - 4s - 8ms/step - accuracy: 0.8646 - loss: 0.2953
Epoch 6/20
468/468 - 3s - 7ms/step - accuracy: 0.8648 - loss: 0.2918
Epoch 7/20
468/468 - 3s - 7ms/step - accuracy: 0.8652 - loss: 0.2922
Epoch 8/20
468/468 - 3s - 7ms/step - accuracy: 0.8648 - loss: 0.2917
Epoch 9/20
468/468 - 3s - 7ms/step - accuracy: 0.8657 - loss: 0.2905
Epoch 10/20
468/468 - 3s - 7ms/step - accuracy: 0.8652 - loss: 0.2900
Epoch 11/20
468/468 - 5s - 11ms/step - accuracy: 0.8651 - loss: 0.2887
Epoch 12/20
468/468 - 4s - 8ms/step - accuracy: 0.8659 - loss: 0.2873
Epoch 13/20
468/468 - 4s - 8ms/step - accuracy: 0.8663 - loss: 0.2871
Epoch 14/20
468/468 - 5s - 10ms/step - accuracy: 0.8652 - loss: 0.2866
Epoch 15/20
468/468 - 3s -

In [21]:
model.save("BiLSTM_only_emojis.h5")

ValueError: Unable to synchronously create dataset (name already exists)

In [22]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

184/184 - 1s - 5ms/step - accuracy: 0.8523 - loss: 0.3754
score: 0.38
acc: 0.85


In [23]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose=0)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 73.92739273927393 %
neg_acc 92.72930648769575 %


In [24]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1/1 - 0s - 378ms/step
negative
